In [45]:
from five9 import Five9
from io import StringIO
import pandas as pd
import datetime
from datetime import datetime, timedelta
import time
from pytz import timezone
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import write_pandas

#Connect in FIVE9
def authenticate(username, pwd):
    auth = Five9('tlgjustin', 'WyzwZMa41s42CXuBu2a8bFGbDKzzJGAi')
    config = auth.configuration
    return (auth, config)

f9, config = authenticate("username", "password")

#Set timezone
now_utc = datetime.now(timezone('UTC'))
now_eastern = now_utc.astimezone(timezone('US/Eastern'))

#Change days from current time
startreportime = now_eastern - timedelta(days=120)
endreportime = now_eastern - timedelta(days=31)

#Set start and end time for report criteria
starttime = f"{(startreportime):%Y-%m-%d}" + 'T00:00:00.000'
endtime = f"{(endreportime):%Y-%m-%d}" + 'T23:59:59.999'

#Set variables as start and end
start = starttime
end = endtime
print(start)
print(end)

#set criteria using variables
criteria = {'time':{'end':end, 'start':start}}

#set folder, report and target table
folder = "Agent Reports"
report = "Agent Login-Logout"
target_table = 'AGENT_LOGIN_LOGOUT'

#Run the report 
identifier = config.runReport(folderName = folder,
                              reportName = report,
                              criteria = criteria)

#Wait if the report is running 
while(config.isReportRunning(identifier, 5)):
  time.sleep(3)
  
#Retrieve data from execution 
result = config.getReportResultCsv(identifier)

#Convert string to DataFrame 
data = StringIO(result)
df = pd.read_csv(data, sep=",")

#Count Number of records pulled
num_of_rows = len(df)
print(f"The number of rows is {num_of_rows}")

#Set dates used for extraction
df['START_EXTRACTION_DATE'] = start
df['END_EXTRACTION_DATE'] = end

#Connect Snowflake & Insert data into it. 
engine2 = snowflake.connector.connect(
        account = 'irsopih-uyb98241',
        user = 'SPIRES',
        password = 'Olivetti@5!',
        database = 'PRD_PYTHON_SOURCE',
        schema = 'FIVE9',
        warehouse = 'COMPUTE_WH',
        role='SYSADMIN',) 

cs = engine2.cursor()

write_pandas(engine2, df, target_table, auto_create_table=True, overwrite = False)

engine2.close()

2023-06-07T00:00:00.000
2023-09-04T23:59:59.999


C:\Users\SAP\AppData\Local\Temp\ipykernel_12672\253842876.py:61: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data, sep=",")


The number of rows is 59052
